In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

2022-12-22 14:05:10.264952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 14:05:10.549574: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-22 14:05:10.599320: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-22 14:05:10.599376: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
def create_array(num,df):
    array = []
    i = num
    for i in range(i,i+10):
        array.append(df['aX'].iloc[i])
    i = num
    for i in range(i,i+10):
        array.append(df['aY'].iloc[i])
    i = num
    for i in range(i,i+10):
        array.append(df['aZ'].iloc[i])
    i = num
    for i in range(i,i+10):
        array.append(df['gX'].iloc[i])
    i = num
    for i in range(i,i+10):
        array.append(df['gY'].iloc[i])
    i = num
    for i in range(i,i+10):
        array.append(df['gZ'].iloc[i])
    return array

In [3]:
def fill_data(size,df):
    df_new = pd.read_csv('datasets/1x60_d_shots/header.csv')
    i=0
    j=0
    while i <= size:
        df_new = pd.DataFrame(np.insert(df_new.values,j,values=create_array(i,df),axis=0))
        i = i + 10
        j = j+1
    return df_new

In [4]:
def record_capture(capture):
    df_test = pd.read_csv('datasets/test_shots/'+capture+'.csv',header=None)
    df_test.columns = ['aX','aY','aZ','gX','gY','gZ']
    capt = fill_data(len(df_test)-1,df_test)
    capt = np.asarray(capt).astype('float32')
    
    return capt

In [5]:
def shot_name(result):
    shots=[]
    for i in range(0,len(result)):
        if((result[i][0] == 1) & (result[i][1] == 1) & (result[i][2] == 0)):
            shots.append('forehand topspin')
        elif((result[i][0] == 0) & (result[i][1] == 1) & (result[i][2] == 0)):
             shots.append('backhand topspin')
        elif((result[i][0] == 1) & (result[i][1] == 0) & (result[i][2] == 0)):
            shots.append('forehand slice')
        elif((result[i][0] == 0) & (result[i][1] == 0) & (result[i][2] == 0)):
             shots.append('backhand slice')
        elif((result[i][2] == 1)):
            shots.append('bad form')
    return shots

In [6]:
def game_shots(result):
    list=[]
    for x in shot_name(result):
        list.append(x)
    return list

In [7]:
def game_stats(game):
    ft=fs=bt=bs=bf=0
    list=[]
    for i in range(0,len(game)):
        if(game[i] == 'forehand topspin'):
            ft = ft+1
        elif(game[i] == 'forehand slice'):
            fs = fs+1
        elif(game[i] == 'backhand topspin'):
            bt = bt+1
        elif(game[i] == 'backhand slice'):
            bs = bs+1
        elif(game[i] == 'bad form'):
            bf = bf+1
    list.append(ft)
    list.append(fs)
    list.append(bt)
    list.append(bs)
    list.append(bf)
    return list

In [8]:
def dict_game_stats(shots_list):
    list = []
    list = shots_list
    list = game_stats(list)
    data = {}
    data['forehand topspin']=list[0]
    data['forehand slice']=list[1]
    data['backhand topspin']=list[2]
    data['backhand slice']=list[3]
    data['bad form']=list[4]
    return data

In [9]:
model = tf.keras.models.load_model('models/tt_project.h5')

2022-12-22 14:05:16.795798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-22 14:05:16.796050: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-22 14:05:16.796083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ali-Inspiron-15-3520): /proc/driver/nvidia/version does not exist
2022-12-22 14:05:16.797134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
import gradio as gr
def greet(file_name):
    
    capture = record_capture(file_name)
    result = model.predict(capture)
    result = np.rint(result)
    data = dict_game_stats(game_shots(result))
    myplot = plt.figure(figsize=(9,4))
    plt.bar(data.keys(), data.values())
    return myplot,pd.DataFrame(shot_name(result))
iface = gr.Interface(fn=greet, inputs="text", outputs=["plot","dataframe"])
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/ali/anaconda3/lib/python3.9/site-packages/gradio/routes.py", line 321, in run_predict
    output = await app.blocks.process_api(
  File "/home/ali/anaconda3/lib/python3.9/site-packages/gradio/blocks.py", line 1015, in process_api
    result = await self.call_function(fn_index, inputs, iterator, request)
  File "/home/ali/anaconda3/lib/python3.9/site-packages/gradio/blocks.py", line 856, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/ali/anaconda3/lib/python3.9/site-packages/anyio/to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "/home/ali/anaconda3/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "/home/ali/anaconda3/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "/tm

1/1 [==============================] - 0s 29ms/step


In [11]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 331.6 kB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 656.4 kB/s eta 0:00:00 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 351.1 kB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 371.6 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 629.3 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 349.5 kB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 639.1 kB/s eta 0:00:001m665.4 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 358.7 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 623.0 kB/s eta 0:00:001m698.7 kB/s eta 0:00:01
     ━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:000 MB/s eta 0:00:01
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4693 sha256=254421453c23a06fae4b06814aa5a53216830710191cecc70a93699c5ae77de4
  Stored in directory: /home/ali/.cache/pip/wheels/91/e2/96/f676aa08bfd789328c6576cd0f1fde4a3d686703bb0c247697
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31671 sha256=742576a0d1caa557610e2997cf7e8c68729866f219246c21fef59f993108006e
  Stored in directory: /home/ali/.cache/pip/wheels/fe/04/d1/a10661cc45f03c3cecda50deb2d2c22f57b4e84a75b2a5987e
Successfully built ffmpy python-multipart
Note: you may need to restart the kernel to use updated packages.
